In [2]:
import random
import math
import statistics
import matplotlib.pyplot as plt
import numpy as np
from pprint import pprint
from deap import base, creator, tools, algorithms

## Iterované vězňovo dilema

In [3]:
def scores(move1, move2):
    # 1 = betray, 0 = not betray

    scores = (0, 0)

    if (move1 == 1) and (move2 == 1):
        scores = (2, 2)

    if (move1 == 1) and (move2 == 0):
        scores = (0, 3)

    if (move1 == 0) and (move2 == 1):
        scores = (3, 0)

    if (move1 == 0) and (move2 == 0):
        scores = (1, 1)

    return scores

def play(player1_strategy, player2_strategy, num_steps):

    score1 = 0
    score2 = 0

    history1 = []
    history2 = []

    for i in range(num_steps):
        move1 = player1_strategy(history1, history2)
        move2 = player2_strategy(history2, history1)

        s1, s2 = scores(move1, move2)
        score1 += s1
        score2 += s2

        history1.append(move1)
        history2.append(move2)

    return score1, score2


## Special cases

#### initial_betrayal
betraying when playing the first round (no history)

#### cautious_vs_sorry
betraying after successfully getting away with it (3, 0) case

#### insidious_vs_peaceful
betraying after the opponent hasn't betrayed me twice

#### revenge_vs_chance
betraying after being betrayed three times in a row

## Cases based on score
winning significantly, loosing significantly or the average case

In [22]:
def strategy(initial_betrayal, cautious_vs_sorry, insidious_vs_peaceful, revenge_vs_chance, winning, loosing, average):

    def configured_strategy(my_history, opponents_history):
        my_move = 0
        if len(my_history) == 0:
            my_move = initial_betrayal
        elif my_history[-1] == 1 and opponents_history[-1] == 0:
            my_move = cautious_vs_sorry
        elif opponents_history[-2:] == [0, 0]:
            my_move = insidious_vs_peaceful
        elif opponents_history[-3:] == [1, 1, 1]:
            my_move = revenge_vs_chance
        else:
            my_score, opponents_score = tuple(sum(pair) for pair in zip(*[scores(x, y) for x, y in zip(my_history, opponents_history)]))
            if my_score / (opponents_score + 0.00001) > 1.25:
                my_move = winning
            elif my_score / (opponents_score + 0.00001) < 0.75:
                my_move = loosing
            else:
                my_move = average      
        return my_move

    return configured_strategy
    

In [27]:
training_strategy = [1, 1, 0, 0, 1, 0, 1]

In [28]:
def fitness(my_strategy):
    opponent_strategy = training_strategy
    my_strategy = strategy(*my_strategy)
    opponent_strategy = strategy(*opponent_strategy)
    my_score, opponent_score = play(my_strategy, opponent_strategy, 10)
    return (my_score,)
    

In [29]:
creator.create("FitnessMax", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)
toolbox = base.Toolbox()
toolbox.register("attr_int", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attr_int, n=7)
toolbox.register("population", tools.initRepeat, list, toolbox.individual) 

toolbox.register("evaluate", fitness)

toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

population = toolbox.population(n=10)
NGEN = 50            
CXPB = 0.7           
MUTPB = 0.2       

finalpopulation, logbook = algorithms.eaSimple(population, toolbox, cxpb=CXPB, mutpb=MUTPB, ngen=NGEN)

gen	nevals
0  	10    
1  	9     
2  	8     
3  	9     
4  	9     
5  	4     
6  	7     
7  	6     
8  	8     
9  	6     
10 	8     
11 	9     
12 	10    
13 	8     
14 	6     
15 	8     
16 	10    
17 	6     
18 	4     
19 	8     
20 	5     
21 	9     
22 	9     
23 	6     
24 	8     
25 	2     
26 	8     
27 	6     
28 	9     
29 	7     
30 	2     
31 	6     
32 	9     
33 	8     
34 	7     
35 	9     
36 	8     
37 	8     
38 	6     
39 	8     
40 	8     
41 	7     
42 	8     
43 	8     
44 	10    
45 	4     
46 	8     
47 	10    
48 	4     
49 	10    
50 	7     


s kým si to má jedinec rozdat, abych ze získaného skóre vypočítal jeho fitness? Aby to mělo výpovědní hodnotu, tak by měli být všechny porovnávány vůči jedné, neměnné strategii.

In [31]:
print(finalpopulation)

[[0, 1, 1, 0, 1, 1, 1], [0, 1, 1, 0, 1, 1, 1], [0, 1, 0, 0, 1, 1, 1], [0, 1, 1, 0, 1, 1, 1], [0, 1, 1, 0, 1, 1, 1], [0, 1, 1, 0, 1, 1, 1], [0, 1, 0, 0, 1, 1, 1], [0, 1, 1, 0, 1, 1, 1], [0, 1, 0, 0, 1, 1, 1], [0, 1, 0, 0, 1, 1, 1]]
